In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import sys
import os
sys.path.append("..")

from mle_logging import merge_seed_logs, merge_config_logs, load_log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Merge logs into `meta_log.hdf5`

In [8]:
exp_dir = "../experiments/ARS/ant"

def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

sub_dirs = get_immediate_subdirectories(exp_dir)
len(sub_dirs)

100

In [9]:
for s_dir in sub_dirs:
    # Merge different random seeds for each config into separate .hdf5 file
    if not os.path.exists(os.path.join(exp_dir, s_dir, "logs/log.hdf5")):
        try:
            merge_seed_logs(os.path.join(exp_dir, s_dir, "logs/log.hdf5"), 
                            os.path.join(exp_dir, s_dir))
        except Exception:
            print(Exception, os.path.join(exp_dir, s_dir, "logs/log.hdf5"))
    else:
        print("Exists", os.path.join(exp_dir, s_dir, "logs/log.hdf5"))

Exists ../experiments/ARS/ant/b_1_eval_2/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_5/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_4/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_3/logs/log.hdf5
<class 'Exception'> ../experiments/ARS/ant/b_1_eval_11/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_8/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_6/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_1/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_10/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_0/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_7/logs/log.hdf5
Exists ../experiments/ARS/ant/b_1_eval_9/logs/log.hdf5


In [15]:
# Aggregate the different merged configuration .hdf5 files into single meta log
merge_config_logs(experiment_dir=exp_dir,
                  all_run_ids=sub_dirs)

# Create `hyper_log.pkl` from `meta_log.hdf5`

In [21]:
meta_log = load_log(exp_dir + "/meta_log.hdf5", True)

metric = "test_perf_strat_mean"

/Users/rob/anaconda3/envs/mle-toolbox/lib/python3.9/site-packages/numpy/lib/function_base.py:4650: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  arr.partition(


In [40]:
import numpy as np
from mle_logging import load_config
all_results = {}
for i, e_id in enumerate(meta_log.eval_ids):
    config = load_config(os.path.join(exp_dir, e_id + ".yaml"), True)
    lrate = config.train_config.es_params.lrate_init
    sigma = config.train_config.es_params.sigma_init
    best_perf = np.max(meta_log[e_id].stats[metric].mean)
    all_results[i] = {"run_id": e_id,
                      "params": {"es_params/lrate_init": lrate,
                                 "es_params/sigma_init": sigma},
                      metric: best_perf}

In [41]:
from mle_toolbox.utils import save_pkl_object
save_pkl_object(all_results, os.path.join(exp_dir, "hyper_log.pkl"))